In [2]:
import os.path as osp
from utils import *
import mmcv
import torch
import wandb
from mmcv import Config
from mmcv.parallel import MMDistributedDataParallel
from mmdet.apis import set_random_seed, train_detector
from mmdet.apis import init_detector, inference_detector, show_result_pyplot
from algo2_result_to_aligned_result import algo2_result_to_aligned_result
# Let's take a look at the dataset image
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from xml_to_np import xml_to_np
from AP import calc_iou_individual, get_single_image_results
from soft_nms import py_cpu_softnms
from dbscan_result_to_aligned_result import dbscan_result_to_aligned_result
from AP import calc_iou_individual
import xml.etree.ElementTree as ET
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd
from utils import print_LC

In [3]:
checkpoint_file = '/home/aiarhipov/centernet/exps/16_paper_params_dla34_batch8/epoch_150.pth'

config_file = "/home/aiarhipov/centernet/exps/16_paper_params_dla34_batch8/config.py"
cfg = Config.fromfile(config_file)

set_random_seed(0, deterministic=False)

dataset = build_dataset(cfg.data.test)

model = build_detector(cfg.model)
model.CLASSES = dataset.CLASSES

model = init_detector(config_file, checkpoint_file, device='cuda:6')

loading annotations into memory...
Done (t=10.42s)
creating index...
index created!
load checkpoint from local path: /home/aiarhipov/centernet/exps/16_paper_params_dla34_batch8/epoch_150.pth


2023-05-04 22:07:41,934 - root - INFO - ModulatedDeformConvPack neck.deconv_layers.0.conv is upgraded to version 2.
2023-05-04 22:07:42,112 - root - INFO - ModulatedDeformConvPack neck.deconv_layers.2.conv is upgraded to version 2.
2023-05-04 22:07:42,225 - root - INFO - ModulatedDeformConvPack neck.deconv_layers.4.conv is upgraded to version 2.


In [4]:
conf_threshold = 0.5
iou_threshold = 0.9
res = []
# for idx in tqdm(range(int(len(dataset)))):
idx = 7
anno = dataset.get_ann_info(idx=idx)
boxes = anno["bboxes"]
segm_path = anno["seg_map"]
xml_path = f"/home/aiarhipov/datasets/WTW-dataset/test/xml/{segm_path[:-4]}.xml"
img_path = f"/home/aiarhipov/datasets/WTW-dataset/test/images/{segm_path[:-4]}.jpg"

gt_boxes = xml_to_np(xml_path)
# print(f"gt_boxes[0] = {gt_boxes[:5]}")
pred = inference_detector(model, img_path)[0]
# print(f"pred[0] = {pred[:5]}")
conf_indexes = py_cpu_softnms(pred[:, :4], pred[:, 4], thresh=0.475, method=2)
# print(f"conf_indexes = {conf_indexes}")
conf_pred = pred[conf_indexes]
# print(f"conf_pred[0] = {conf_pred[:5]}")
if conf_pred.shape[0] > 1:
    conf_pred = dbscan_result_to_aligned_result([conf_pred])
    if conf_pred:
        conf_pred = conf_pred[0]
        print(gt_boxes[:, :4].shape)
        print(conf_pred[:, :4].shape)

(92, 4)
(87, 4)


In [5]:
# show_result_pyplot(model, img_path, [conf_pred[:, :4]], score_thr = 0)

In [6]:
def LC_for_dbscan_algo(pred):
    res = np.pad(pred, ((0,0),(0,4)), mode="constant", constant_values=np.nan)
    # left_edge.append(((box[0], box[1]), (box[0], box[3])))
    # right_edge.append(((box[2], box[1]), (box[2], box[3])))
    # up_edge.append(((box[0], box[1]), (box[2], box[1])))
    # down_edge.append(((box[0], box[3]), (box[2], box[3])))
    x = np.unique(np.hstack((pred[:, 0], pred[:, 2])))
    y = np.unique(np.hstack((pred[:, 1], pred[:, 3])))
    for idx, box in enumerate(pred):
        startrow = np.where(np.isclose(y, box[1]))[0][0]
        endrow = np.where(np.isclose(y, box[3]))[0][0] - 1
        startcolumn = np.where(np.isclose(x, box[0]))[0][0]
        endcolumn = np.where(np.isclose(x, box[2]))[0][0] - 1
        res[idx, -1] = endcolumn
        res[idx, -2] = startcolumn
        res[idx, -3] = endrow
        res[idx, -4] = startrow
    return res
    
# print_LC(img_path, LC_for_dbscan_algo(conf_pred))

In [7]:
conf_pred = LC_for_dbscan_algo(conf_pred)
matched_gt = np.insert(gt_boxes, 4, np.nan, axis=1)
for idx, g in enumerate(gt_boxes):
    
    iou = pd.Series([calc_iou_individual(p[:4], g[:4]) for p in conf_pred])
    max_idx = iou.argmax()
    max_idx = max_idx if iou[max_idx] > iou_threshold else np.nan
    matched_gt[idx, 4] = max_idx

In [8]:
print_LC(img_path, np.insert(conf_pred, 9, range(conf_pred.shape[0]), axis=1), out_path="example_with_bounding_boxes_pred.jpg")
print_LC(img_path, np.insert(matched_gt, 9, range(matched_gt.shape[0]), axis=1), out_path="example_with_bounding_boxes_gt.jpg")

In [9]:
def adjacency_relation_per_image_dbscan(gt, pred, iou_threshold=0) -> dict:
    """_summary_

    Parameters
    ----------
    gt : shape(n, 4) xmin, ymin, xmax, ymax, [match], startrow, endrow, startcol, endcol
    gt : shape(n, 4)   0 ,  1  ,  2  ,   3 ,    4   ,     5   ,    6  ,     7   ,   8
        
    pred : shape(n, 5) xmin, ymin, xmax, ymax, confidence, [startrow, endrow, startcol, endcol]
    pred : shape(n, 5)   0 ,  1  ,  2  ,   3 ,       4   ,     5    ,    6  ,     7   ,   8
    
    """
    pred = LC_for_dbscan_algo(pred)
    matched_gt = np.insert(gt, 4, np.nan, axis=1)
    for idx, g in enumerate(gt):
        iou = pd.Series([calc_iou_individual(p[:4], g[:4]) for p in pred])
        max_idx = iou.argmax()
        max_idx = max_idx if iou[max_idx] > iou_threshold else np.nan
        matched_gt[idx, 4] = max_idx

    tp = 0
    gt_lines = 0
    pred_lines = 0
    for idx, g in enumerate(matched_gt):
        if not np.isnan(g[4]):
            anchor_pred_idx = int(g[4])
            gt_right_idx,  = np.where((matched_gt[:,7] == g[8]+1) & (matched_gt[:,5] <= g[5]) & (matched_gt[:,6] >= g[5]))
            if gt_right_idx:
                gt_lines+=1 
                gt_right_idx = gt_right_idx[0]
                pred_right_idx, = np.where((pred[:,7] == pred[anchor_pred_idx, 8] + 1) & (pred[:,5] <= pred[anchor_pred_idx, 5])& (pred[:,6] >= pred[anchor_pred_idx, 5]))
                if pred_right_idx:
                    pred_lines+=1
                    pred_right_idx = pred_right_idx[0]
                    if pred_right_idx == matched_gt[gt_right_idx, 4]:
                        tp += 1


            gt_down_idx, = np.where((matched_gt[:,5] == g[6]+1) & (matched_gt[:,7] <= g[7]) & (matched_gt[:,8] >= g[7]))
            if gt_down_idx:
                gt_lines+=1 
                gt_down_idx = gt_down_idx[0]
                pred_down_idx, = np.where((pred[:,5] == pred[anchor_pred_idx, 6] + 1) & (pred[:,7] <= pred[anchor_pred_idx, 7])& (pred[:,8] >= pred[anchor_pred_idx, 7]) )   
                if pred_down_idx:
                    pred_lines+=1
                    pred_down_idx = pred_down_idx[0]
                    if pred_down_idx == matched_gt[gt_down_idx, 4]:
                        tp += 1

    return {"precision":tp/pred_lines, "recall":tp/gt_lines, "tp":tp, "gt_lines":gt_lines, "pred_lines":pred_lines}
adjacency_relation_per_image_dbscan(gt_boxes, conf_pred, iou_threshold=0.5)

/tmp/ipykernel_4107/1968879673.py:40: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if gt_down_idx:
/tmp/ipykernel_4107/1968879673.py:32: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if pred_right_idx:
/tmp/ipykernel_4107/1968879673.py:28: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if gt_right_idx:


{'precision': 1.0,
 'recall': 0.9787234042553191,
 'tp': 138,
 'gt_lines': 141,
 'pred_lines': 138}

In [10]:
conf_threshold = 0.5
iou_threshold = 0.9
res = []
for idx in tqdm(range(int(len(dataset)))):
    anno = dataset.get_ann_info(idx=idx)
    boxes = anno["bboxes"]
    segm_path = anno["seg_map"]
    xml_path = f"/home/aiarhipov/datasets/WTW-dataset/test/xml/{segm_path[:-4]}.xml"
    img_path = f"/home/aiarhipov/datasets/WTW-dataset/test/images/{segm_path[:-4]}.jpg"
    
    gt_boxes = xml_to_np(xml_path)
    pred = inference_detector(model, img_path)[0]
    conf_indexes = py_cpu_softnms(pred[:, :4], pred[:, 4], thresh=0.475, method=2)
    conf_pred = pred[conf_indexes]
    if conf_pred.shape[0] > 1:
        conf_pred = dbscan_result_to_aligned_result([conf_pred])
        if conf_pred:
            conf_pred = conf_pred[0]
            metrics = adjacency_relation_per_image_dbscan(gt_boxes, conf_pred)
            res.append(metrics)

Precision = np.mean([d["precision"] for d in res])
Recall = np.mean([d["recall"] for d in res])

print(f"Precision = {Precision}")
print(f"Recall = {Recall}")


  0%|          | 0/3611 [00:00<?, ?it/s]

/tmp/ipykernel_4107/1968879673.py:16: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  iou = pd.Series([calc_iou_individual(p[:4], g[:4]) for p in pred])


ValueError: attempt to get argmax of an empty sequence